## **KMP (Knuth-Morris-Pratt)**

In [ ]:
# kmp_table
pattern = 'ababbab'

table = [0 for _ in range(len(pattern))]
i = 0
for j in range(1, len(pattern)):
    while i > 0 and pattern[i] != pattern[j]:
        i = table[i - 1]

    if pattern[i] == pattern[j]:
        i += 1
        table[j] = i
print(table)
# [0, 0, 1, 2, 0, 1, 2]

In [ ]:
# kmp
def kmp(all_string, pattern):
    #kmp_table
    table = [0 for _ in range(len(pattern))]
    i = 0
    for j in range(1, len(pattern)):
        while i > 0 and pattern[i] != pattern[j]:
            i = table[i - 1]

        if pattern[i] == pattern[j]:
            i += 1
            table[j] = i

    # kmp
    result = []
    i = 0
    for j in range(len(all_string)):
        while i > 0 and pattern[i] != all_string[j]:
            i = table[i - 1]

        if pattern[i] == all_string[j]:
            i += 1
            if i == len(pattern):
                result.append(j - i + 1)
                i = table[i - 1]

    return result

print(kmp('xabxxbaxbaxbaxbaxabxbaxbabx', 'abx')) # [1, 17, 24]
print(kmp('abababab', 'abab')) # [0, 2, 4]

## **Trie**

In [ ]:
class Node(object):
    def __init__(self, key, data=None):
        self.key = key
        self.data = data
        self.children = {}

class Trie:
    def __init__(self):
        self.head = Node(None)

    def insert(self, string):
        current_node = self.head

        for char in string:
            if char not in current_node.children:
                current_node.children[char] = Node(char)
            current_node = current_node.children[char]
        current_node.data = string

    def search(self, string):
        current_node = self.head

        for char in string:
            if char in current_node.children:
                current_node = current_node.children[char]
            else:
                return False
            
        if current_node.data:
            return True
        else:
            return False
        
    def starts_with(self, prefix):
        current_node = self.head
        words = []

        for p in prefix:
            if p in current_node.children:
                current_node = current_node.children[p]
            else:
                return None
            
        current_node = [current_node]
        next_node = []
        while True:
            for node in current_node:
                if node.data:
                    words.append(node.data)
                next_node.extend(list(node.children.values()))
            if len(next_node) != 0:
                current_node = next_node
                next_node = []
            else:
                break
        return words

In [ ]:
trie = Trie()
word_list = ["frodo", "front", "firefox", "fire"]
for word in word_list:
    trie.insert(word)

print(trie.search("friend")) # False
print(trie.search("frodo")) # True
print(trie.search("fire")) # True

print(trie.starts_with("fire")) # ['fire', 'firefox']
print(trie.starts_with("fro")) # ['frodo', 'front']
print(trie.starts_with("jimmy")) # None
print(trie.starts_with("f")) # ['fire', 'frodo', 'front', 'firefox']

## **Aho-Corasick**

In [ ]:
from collections import deque

class Node:
    def __init__(self, key=None, data=None):
        self.key = key # 현재 노드의 문자
        self.data = data # 문자열이 끝나는 노드라면 저장되는 패턴
        self.children = {} # 자식 노드 (Trie 구조)
        self.fail = None # 실패 링크 (패턴 불일치 시 이동할 노드)
        self.output = [] # 매칭된 패턴 리스트 (현재 노드까지 도달했을 때 완성되는 문자열들)

class AhoCorasick:
    def __init__(self):
        self.root = Node()

    def insert(self, string): # Trie에 패턴 문자열 삽입
        current_node = self.root
        for char in string:
            # 자식 노드가 없으면 새로 추가
            if char not in current_node.children:
                current_node.children[char] = Node(char)
            current_node = current_node.children[char]

        # 문자열 끝 노드에 패턴 저장
        current_node.data = string
        current_node.output.append(string)

    def build_fail_links(self):
        # BFS로 모든 노드에 대해 실패(fail) 링크 설정
        q = deque()
        self.root.fail = self.root

        for child in self.root.children.values():
            child.fail = self.root
            q.append(child)

        while q:
            current_node = q.popleft()

            # 현재 노드의 모든 자식 노드 처리
            for key, child in current_node.children.items():
                q.append(child)

                # 실패 링크를 따라가면서 현재 key와 매칭되는 노드를 찾음
                fail_node = current_node.fail
                while fail_node != self.root and key not in fail_node.children:
                    fail_node = fail_node.fail

                # 매칭되는 문자가 있으면 실패 링크 설정, 없으면 루트로
                if key in fail_node.children:
                    child.fail = fail_node.children[key]
                else:
                    child.fail = self.root

                # 실패 링크가 가진 output을 상속받아 연결 (ex. "he" → "e")
                child.output += child.fail.output

    def search(self, text):
        # 입력된 텍스트에서 모든 패턴을 검색
        # 반환: (패턴 시작 인덱스, 매칭된 패턴) 리스트
        current_node = self.root
        matches = []

        # 텍스트를 한 글자씩 순회
        for i, char in enumerate(text):
            # 현재 문자와 일치하는 자식이 없으면 실패 링크를 따라감
            while current_node != self.root and char not in current_node.children:
                current_node = current_node.fail

            # 매칭되는 자식이 있으면 이동, 없으면 루트로
            if char in current_node.children:
                current_node = current_node.children[char]
            else:
                current_node = self.root

            # output 리스트가 비어있지 않으면 매칭된 패턴이 있다는 의미
            if current_node.output:
                for pattern in current_node.output:
                    start_idx = i - len(pattern) + 1
                    matches.append((start_idx, pattern))
        
        return matches

In [ ]:
aho = AhoCorasick()
patterns = ['he', 'she', 'his', 'hers']

for p in patterns:
    aho.insert(p)

aho.build_fail_links()

text = 'ushers'
result = aho.search(text)
for idx, word in result:
    print(f'Pattern: {word} (idx: {idx})')
# Pattern: she (idx: 1)
# Pattern: he (idx: 2)
# Pattern: hers (idx: 2)

16916번 부분 문자열 <span style="color:green">성공</span> - 2025.09.01

In [ ]:
s = input()
p = input()

if p in s:
    print(1)
else:
    print(0)

1786번 찾기 <span style="color:green">성공</span> - 2025.09.02

In [ ]:
def kmp(t, p):
    table = [0 for _ in range(len(p))]
    i = 0
    for j in range(1, len(p)):
        while i > 0 and p[i] != p[j]:
            i = table[i - 1]

        if p[i] == p[j]:
            i += 1
            table[j] = i

    i = 0
    for j in range(len(t)):
        while i > 0 and p[i] != t[j]:
            i = table[i - 1]

        if p[i] == t[j]:
            i += 1
            if i == len(p):
                ans.append(j - i + 2)
                i = table[i - 1]

t = input()
p = input()

ans = []
kmp(t, p)
print(len(ans))
print(*ans)

1305번 광고 <span style="color:green">성공</span> - 2025.09.03

In [ ]:
def kmp_table(p):
    table = [0 for _ in range(len(p))]
    i = 0
    for j in range(1, len(p)):
        while i > 0 and p[i] != p[j]:
            i = table[i - 1]
        
        if p[i] == p[j]:
            i += 1
            table[j] = i
    return table

l = int(input())
s = input()

print(l - kmp_table(s)[-1])

1701번 Cubeditor <span style="color:green">성공</span> - 2025.09.04

In [ ]:
def kmp(t, p):
    table = [0 for _ in range(len(p))]
    i = 0
    for j in range(1, len(p)):
        while i > 0 and p[i] != p[j]:
            i = table[i - 1]

        if p[i] == p[j]:
            i += 1
            table[j] = i

    result = []
    i = 0
    for j in range(len(t)):
        while i > 0 and p[i] != t[j]:
            i = table[i - 1]

        if p[i] == t[j]:
            i += 1
            if i == len(p):
                result.append(j - i + 2)
                i = table[i - 1]
    return len(result)

s = input()

ans = 0
for i in range(1, len(s)):
    cursor = 0
    flag = False
    while cursor + i <= len(s):
        if kmp(s, s[cursor:cursor + i]) >= 2:
            flag = True
            ans = max(ans, i)
            break
        cursor += 1
    
    if not flag:
        break
print(ans)

14425번 문자열 집합 <span style="color:green">성공</span> - 2025.09.05

In [ ]:
# import sys
# input = sys.stdin.readline

n, m = map(int, input().split())
word = set()
for _ in range(n):
    word.add(input())

ans = 0
for _ in range(m):
    if input() in word:
        ans += 1
print(ans)

14426번 접두사 찾기 <span style="color:red">실패</span> - 2025.09.06

In [ ]:
import sys
from bisect import bisect
input = sys.stdin.readline

n, m = map(int, input().split())
words = sorted([input().rstrip() for _ in range(n)])

ans = 0
for _ in range(m):
    prefix = input().rstrip()

    idx = min(n - 1, bisect(words, prefix))
    if words[idx].startswith(prefix):
        ans += 1
    elif words[idx - 1].startswith(prefix):
        ans += 1
print(ans)

13505번 두 수 XOR <span style="color:green">성공</span> - 2025.09.07

In [ ]:
class Node(object):
    def __init__(self, key, data=None):
        self.key = key
        self.data = data
        self.children = {}

class Trie:
    def __init__(self):
        self.head = Node(None)

    def insert(self, string):
        current_node = self.head

        for char in string:
            if char not in current_node.children:
                current_node.children[char] = Node(char)
            current_node = current_node.children[char]
        current_node.data = string
    
    def find_bin(self, bin_num):
        current_node = self.head

        for i in range(len(bin_num)):
            if bin_num[i] == '0':
                if '1' in current_node.children:
                    current_node = current_node.children['1']
                else:
                    current_node = current_node.children['0']
            elif bin_num[i] == '1':
                if '0' in current_node.children:
                    current_node = current_node.children['0']
                else:
                    current_node = current_node.children['1']

        return current_node.data

n = int(input())
a = list(map(int, input().split()))
max_a_len = len(bin(max(a))) - 2

bin_a = [format(n, f'0{max_a_len}b') for n in a]

trie = Trie()
for i in range(n):
    trie.insert(bin_a[i])

ans = 0
for i in range(n):
    optimal = trie.find_bin(bin_a[i])
    ans = max(ans, int(bin_a[i], 2) ^ int(optimal, 2))
print(ans)

9250번 문자열 집합 판별 <span style="color:green">성공</span> - 2025.09.08

In [ ]:
from collections import deque

class Node:
    def __init__(self, key=None, data=None):
        self.key = key
        self.data = data
        self.children = {}
        self.fail = None
        self.output = []

class AhoCorasick:
    def __init__(self):
        self.root = Node()

    def insert(self, string):
        current_node = self.root
        for char in string:
            if char not in current_node.children:
                current_node.children[char] = Node(char)
            current_node = current_node.children[char]

        current_node.data = string
        current_node.output.append(string)

    def build_fail_links(self):
        q = deque()
        self.root.fail = self.root

        for child in self.root.children.values():
            child.fail = self.root
            q.append(child)

        while q:
            current_node = q.popleft()

            for key, child in current_node.children.items():
                q.append(child)

                fail_node = current_node.fail
                while fail_node != self.root and key not in fail_node.children:
                    fail_node = fail_node.fail

                if key in fail_node.children:
                    child.fail = fail_node.children[key]
                else:
                    child.fail = self.root

                child.output += child.fail.output

    def search(self, text):
        current_node = self.root
        matches = []

        for i, char in enumerate(text):
            while current_node != self.root and char not in current_node.children:
                current_node = current_node.fail

            if char in current_node.children:
                current_node = current_node.children[char]
            else:
                current_node = self.root

            if current_node.output:
                for pattern in current_node.output:
                    start_idx = i - len(pattern) + 1
                    matches.append((start_idx, pattern))
        
        return matches
    
n = int(input())
p = [input() for _ in range(n)]
q = int(input())
t = [input() for _ in range(q)]

aho = AhoCorasick()

for i in range(n):
    aho.insert(p[i])

aho.build_fail_links()

for i in range(q):
    if aho.search(t[i]):
        print('YES')
    else:
        print('NO')

In [ ]:

aho = AhoCorasick()
patterns = ['he', 'she', 'his', 'hers']

for p in patterns:
    aho.insert(p)

aho.build_fail_links()

text = 'ushers'
result = aho.search(text)
for idx, word in result:
    print(f'Pattern: {word} (idx: {idx})')
# Pattern: she (idx: 1)
# Pattern: he (idx: 2)
# Pattern: hers (idx: 2)

10256번 돌연변이 <span style="color:green">실패</span> - 2025.09.09

In [ ]:
from collections import deque
import sys
input = sys.stdin.readline

char_idx = {'A': 0, 'C': 1, 'G': 2, 'T': 3}

class Node:
    # __slots__ 기능
    # 일반적으로 Python 클래스는 속성을 동적으로 
    # 추가할 수 있도록 내부에 __dict__를 갖고 있음
    # 각 객체당 메모리가 꽤 많이 사용
    # __slots__ 를 사용하면
    # __dict__를 만들지 않고 고정된 슬롯 공간만 할당
    # 노드 하나당 메모리 사용량이 훨씬 감소
    __slots__ = ['children', 'fail', 'is_end']
    def __init__(self):
        self.children = [None] * 4
        self.fail = None
        self.is_end = False

class AhoCorasick:
    def __init__(self):
        self.root = Node()

    def insert(self, string):
        current_node = self.root
        for char in string:
            idx = char_idx[char]
            if current_node.children[idx] is None:
                current_node.children[idx] = Node()
            current_node = current_node.children[idx]
        current_node.is_end = True

    def build_fail_links(self):
        q = deque()
        self.root.fail = self.root

        for child in self.root.children:
            if child:
                child.fail = self.root
                q.append(child)

        while q:
            current_node = q.popleft()

            for i in range(4):
                child = current_node.children[i]
                if not child:
                    continue

                fail_node = current_node.fail
                while fail_node != self.root and fail_node.children[i] is None:
                    fail_node = fail_node.fail

                if fail_node.children[i]:
                    child.fail = fail_node.children[i]
                else:
                    child.fail = self.root

                if child.fail.is_end:
                    child.is_end = True

                q.append(child)

    def search(self, text):
        current_node = self.root
        cnt = 0

        for char in text:
            idx = char_idx[char]
            while current_node != self.root and current_node.children[idx] is None:
                current_node = current_node.fail
            if current_node.children[idx]:
                current_node = current_node.children[idx]
            else:
                current_node = self.root

            if current_node.is_end:
                cnt += 1

        return cnt

t = int(input())
for _ in range(t):
    n, m = map(int, input().split())

    dna = input().strip()
    marker = list(input().strip())

    aho = AhoCorasick()
    aho.insert(''.join(marker))

    for left in range(m - 1):
        for right in range(left + 2, m + 1):
            tmp = marker[:]
            tmp[left:right] = tmp[left:right][::-1]
            aho.insert(''.join(tmp))

    aho.build_fail_links()

    ans = aho.search(dna)
    print(ans)